In [2]:
import pandas as pd
import sqlite3
import os
import datetime
import re
import numpy as np
import sklearn
import functools

In [3]:
post = pd.read_excel(f"xxxx/posts.xlsx",sheet_name='Posts')
comment = pd.read_excel(f"xxxx/posts.xlsx",sheet_name='Comments')
answer = pd.read_excel(f"xxxx/posts.xlsx",sheet_name='Answers')

rq = pd.read_excel(f"xxxx/post-integrated.xlsx")
lda = pd.read_excel(f'xxxx/post-membership.xlsx')

lda = lda[['Id','Title','Text','Topic 1 Perc Contrib',
    'Topic 2 Perc Contrib','Topic 3 Perc Contrib',
    'Topic 4 Perc Contrib','Topic 5 Perc Contrib','Topic 6 Perc Contrib']].rename(columns={'Id':'Id','Title':'Title','Text':'Text',
    'Topic 1 Perc Contrib':'Coding','Topic 2 Perc Contrib':'Energy',
    'Topic 3 Perc Contrib':'Device','Topic 4 Perc Contrib':'Monitoring',
    'Topic 5 Perc Contrib':'CS','Topic 6 Perc Contrib':'DT'})
pd.to_numeric(lda['Coding'])
pd.to_numeric(lda['Energy'])
pd.to_numeric(lda['Device'])
pd.to_numeric(lda['Monitoring'])
pd.to_numeric(lda['CS'])
pd.to_numeric(lda['DT'])
# thrs = 0.318800002336502102
thrs = 0.348100006

In [100]:
def check_tags(df, t):
    dftag = pd.DataFrame()
    for a, tag in df.iterrows():
        if t in re.findall(r'(?<=\<)(.*?)(?=\>)',tag['Tags']):
            dftag = pd.concat([dftag,tag.to_frame().T])
    dftag.fillna(0, inplace=True)
    return dftag


In [5]:
dump_path='/Users/xxxx'
dump_database_name='title-body.db'
db = sqlite3.connect(os.path.join(dump_path, dump_database_name))

In [4]:
# # create database
# c = db.cursor()
# c.execute('DROP TABLE IF EXISTS Temp')
# db.commit()
# c.execute('Create TABLE IF not EXISTS Temp'+
# ' (Id integer PRIMARY KEY, Theme text, Tags text, QuestionDate datetime, '+
# 'QuestionScore	integer, ViewCount integer, AnswerCount integer, CommentCount integer,'+
# 'CommentDate text, FavoriteCount text, AnswerDate text, AnswerScore text,'+
# ' Coding DECIMAL, Energy DECIMAL, Device DECIMAL, Monitoring DECIMAL, CS DECIMAL, DT DECIMAL)')
# db.commit()
# for i in range(len(rq)):
#     id = rq['Id'][i]
#     theme = rq['Title'][i] +" "+ rq['Body'][i]
#     theme = theme.replace("'","''")
#     fc = rq['FavoriteCount'][i] if not pd.isna(rq['FavoriteCount'][i]) else ''
#     cd = rq['CommentDate'][i] if not pd.isna(rq['CommentDate'][i]) else ''
#     ad = rq['AnswerDate'][i] if not pd.isna(rq['AnswerDate'][i]) else ''
#     anss = rq['AnswerScore'][i] if not pd.isna(rq['AnswerScore'][i]) else ''

#     Coding = lda[lda['Id']==id]['Coding'][i].astype('float64') if not pd.isna(lda[lda['Id']==id]['Coding'][i].astype('float64')) else 0
#     Energy = lda[lda['Id']==id]['Energy'][i].astype('float64') if not pd.isna(lda[lda['Id']==id]['Energy'][i].astype('float64')) else 0
#     Device = lda[lda['Id']==id]['Device'][i].astype('float64') if not pd.isna(lda[lda['Id']==id]['Device'][i].astype('float64')) else 0
#     Monitoring = lda[lda['Id']==id]['Monitoring'][i].astype('float64') if not pd.isna(lda[lda['Id']==id]['Monitoring'][i].astype('float64')) else 0
#     CS = lda[lda['Id']==id]['CS'][i].astype('float64') if not pd.isna(lda[lda['Id']==id]['CS'][i].astype('float64')) else 0
#     DT = lda[lda['Id']==id]['DT'][i].astype('float64') if not pd.isna(lda[lda['Id']==id]['DT'][i].astype('float64')) else 0
    
#     c.execute(f"insert into Temp values('{id}','{theme}','{rq['Tags'][i]}',datetime('{rq['QuestionDate'][i]}'),"+
#     f"{rq['QuestionScore'][i]},{rq['ViewCount'][i]},{rq['AnswerCount'][i]},"+
#     f"{rq['CommentCount'][i]},'{cd}','{fc}',"+
#     f"'{ad}','{anss}',"+
#     f"{Coding}, {Energy}, {Device}, {Monitoring}, {CS}, {DT})")
# db.commit()


In [6]:
pop = {} #popularity score for each id
ov = rq['ViewCount'].mean()
print(ov)
for i in range(len(rq)):
    s= float(rq['QuestionScore'][i]) 
    a= float(rq['AnswerScore'][i]) if not pd.isnull(rq['AnswerScore'][i]) else 0
    cc= float(rq['CommentCount'][i]) 
    f= float(rq['FavoriteCount'][i]) if not pd.isnull(rq['FavoriteCount'][i]) else 0
    v= float(rq['ViewCount'][i])/ov
    p=s+a+cc+f+v
    pop[rq['Id'][i]] = {'s':s,'a':a,'c':cc,'f':f,'v':v,'p':p}

smin = np.array([ d['s'] for d in list(pop.values())]).min()
smax = np.array([ d['s'] for d in list(pop.values())]).max()
amin = np.array([ d['a'] for d in list(pop.values())]).min()
amax = np.array([ d['a'] for d in list(pop.values())]).max()
cmin = np.array([ d['c'] for d in list(pop.values())]).min()
cmax = np.array([ d['c'] for d in list(pop.values())]).max()
fmin = np.array([ d['f'] for d in list(pop.values())]).min()
fmax = np.array([ d['f'] for d in list(pop.values())]).max()
vmin = np.array([ d['v'] for d in list(pop.values())]).min()
vmax = np.array([ d['v'] for d in list(pop.values())]).max()
pmin = np.array([ d['p'] for d in list(pop.values())]).min()
pmax = np.array([ d['p'] for d in list(pop.values())]).max()


for p in pop:
    s = (pop[p]['s']-smin)/(smax-smin)
    a = (pop[p]['a']-amin)/(amax-amin)
    c = (pop[p]['c']-cmin)/(cmax-cmin)
    f = (pop[p]['f']-fmin)/(fmax-fmin)
    v = (pop[p]['v']-vmin)/(vmax-vmin)
    po = s+a+c+f+v
    
    pop[p] = {'s':s,'a':a,'c': c,'f':f, 'v': v,'p':po}#,'p':(pop[p]['p']-pmin)/(pmax-pmin)}
    
# print(pop)
data = pd.read_sql(f'select Id, Theme, Tags, QuestionDate,'+
' Coding, Energy, Device, Monitoring, CS, DT,' +
' QuestionScore, ViewCount, AnswerCount, CommentCount,'+
' CommentDate, FavoriteCount, AnswerDate, AnswerScore from Temp', con=db)



1366.0162436548223


In [11]:
print('Theme: Coding')
row = data[data['Coding']>=thrs]
# row = data[data['Theme'].str.lower().str.contains('batter') | data['Tags'].str.lower().str.contains('batter')]

print('Q: ',len(row))
print('Freq:', round(len(row)/985*100,2), '%')
a =0
s,u=0,0
aa,ac = [],[]
ps,pa,pc,pf,pv,pp = 0.0,0.0,0.0,0.0,0.0,0.0
for index, e in row.iterrows():
    ps += pop[e[0]]['s']
    pa += pop[e[0]]['a']
    pc += pop[e[0]]['c']
    pf += pop[e[0]]['f']
    pv += pop[e[0]]['v']
    pp += pop[e[0]]['p']
    a += int(e[6+6])
    if e[6+6] == 0: #AnswerCount
        u+=1
    elif e[-2] != '': #AnswerDate
        s+=1
        #AnswerDate-QuestionDate
        aa.append((datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
    if e[-4] != '': #CommentDate
        if e[-2] != '':
            #comment date before the date the author accpets the answer
            #CommentDate-AnswerDate
            if (datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds()<0:
                #CommentDate-QuestionDate
                ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
        else:
            #CommentDate-QuestionDate
            ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())

print('A: ',a)
print('A/Q: ', round(a/len(row),2))
print('Without an accpeted answer: ',round(100-s/len(row)*100,2), '%')
# print('Successful: ',round(s/len(row)*100,2), '%')
# print('Ordinary: ',round((len(row)-u-s)/len(row)*100,2), '%')
print('Unsuccessful: ',round(u/len(row)*100,2), '%')
# print('Answered: ',round((len(row)-u)/len(row)*100,2), '%')
# print('Unanswered: ',round(u/len(row)*100,2), '%')
aa = sorted(aa, key=lambda x:x, reverse=True)
ac = sorted(ac, key=lambda x:x, reverse=True)
#remove the highest and lowest //no
# print('Median Answer time: ',round(np.median(aa[1:-1])/3600/12,2), 'days')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600/12,2), 'days')
print('Median Answer time: ',round(np.median(aa)/3600,2), 'Hr')
# print('Median Comment time: ',round(np.median(ac)/3600,2), 'Hr')
print('S: ',round(ps,2))
print('A: ',round(pa,2))
print('C: ',round(pc,2))
print('F: ',round(pf,2))
print('V: ',round(pv,2))
print('P: ',round(pp,2))
print("Answer responses: ",len(aa))

Theme: Coding
Q:  171
Freq: 17.36 %
A:  222
A/Q:  1.3
Without an accpeted answer:  57.89 %
Unsuccessful:  23.98 %
Median Answer time:  3.51 Hr
S:  14.0
A:  5.58
C:  21.78
F:  0.95
V:  1.84
P:  44.16
Answer responses:  72


In [12]:
print('Theme: Energy')

row = data[data['Energy']>=thrs]
# row = data[data['Theme'].str.lower().str.contains('background') | data['Tags'].str.lower().str.contains('background')]

print('Q: ',len(row))
print('Freq:', round(len(row)/985*100,2), '%')
a =0
s,u=0,0
aa,ac = [],[]
ps,pa,pc,pf,pv,pp = 0.0,0.0,0.0,0.0,0.0,0.0
for index, e in row.iterrows():
    ps += pop[e[0]]['s']
    pa += pop[e[0]]['a']
    pc += pop[e[0]]['c']
    pf += pop[e[0]]['f']
    pv += pop[e[0]]['v']
    pp += pop[e[0]]['p']
    a += int(e[6+6])
    if e[6+6] == 0: #AnswerCount
        u+=1
    elif e[-2] != '': #AnswerDate
        s+=1
        #AnswerDate-QuestionDate
        aa.append((datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
    if e[-4] != '': #CommentDate
        if e[-2] != '':
            #comment date before the date the author accpets the answer
            #CommentDate-AnswerDate
            if (datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds()<0:
                #CommentDate-QuestionDate
                ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
        else:
            #CommentDate-QuestionDate
            ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())

print('A: ',a)
print('A/Q: ', round(a/len(row),2))
# print('Successful: ',round(s/len(row)*100,2), '%')
print('Without an accpeted answer: ',round(100-s/len(row)*100,2), '%')
# print('Ordinary: ',round((len(row)-u-s)/len(row)*100,2), '%')
print('Unsuccessful: ',round(u/len(row)*100,2), '%')
# print('Answered: ',round((len(row)-u)/len(row)*100,2), '%')
aa = sorted(aa, key=lambda x:x, reverse=True)
ac = sorted(ac, key=lambda x:x, reverse=True)
#remove the highest and lowest
# print('Median Answer time: ',round(np.median(aa[1:-1])/3600/12,2), 'days')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600/12,2), 'days')
print('Median Answer time: ',round(np.median(aa[1:-1])/3600,2), 'Hr')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600,2), 'Hr')
print('S: ',round(ps,2))
print('A: ',round(pa,2))
print('C: ',round(pc,2))
print('F: ',round(pf,2))
print('V: ',round(pv,2))
print('P: ',round(pp,2))
print("Answer responses: ",len(aa))

Theme: Energy
Q:  201
Freq: 20.41 %
A:  221
A/Q:  1.1
Without an accpeted answer:  68.66 %
Unsuccessful:  25.87 %
Median Answer time:  5.14 Hr
S:  17.65
A:  6.7
C:  12.39
F:  2.06
V:  3.39
P:  42.19
Answer responses:  63


In [16]:
print('Theme: Device')

row = data[data['Device']>=thrs]
# row = data[data['Theme'].str.lower().str.contains('android') | data['Tags'].str.lower().str.contains('android')]

print('Q: ',len(row))
print('Freq:', round(len(row)/985*100,2), '%')
a =0
s,u=0,0
aa,ac = [],[]
ps,pa,pc,pf,pv,pp = 0.0,0.0,0.0,0.0,0.0,0.0
for index, e in row.iterrows():
    ps += pop[e[0]]['s']
    pa += pop[e[0]]['a']
    pc += pop[e[0]]['c']
    pf += pop[e[0]]['f']
    pv += pop[e[0]]['v']
    pp += pop[e[0]]['p']
    a += int(e[6+6])
    if e[6+6] == 0: #AnswerCount
        u+=1
    elif e[-2] != '': #AnswerDate
        s+=1
        #AnswerDate-QuestionDate
        aa.append((datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
    if e[-4] != '': #CommentDate
        if e[-2] != '':
            #comment date before the date the author accpets the answer
            #CommentDate-AnswerDate
            if (datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds()<0:
                #CommentDate-QuestionDate
                ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
        else:
            #CommentDate-QuestionDate
            ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())

print('A: ',a)
print('A/Q: ', round(a/len(row),2))
print('Without an accpeted answer: ',round(100-s/len(row)*100,2), '%')
# print('Successful: ',round(s/len(row)*100,2), '%')
# print('Ordinary: ',round((len(row)-u-s)/len(row)*100,2), '%')
print('Unsuccessful: ',round(u/len(row)*100,2), '%')
# print('Answered: ',round((len(row)-u)/len(row)*100,2), '%')
aa = sorted(aa, key=lambda x:x, reverse=True)
ac = sorted(ac, key=lambda x:x, reverse=True)
#remove the highest and lowest
# print('Median Answer time: ',round(np.median(aa[1:-1]),2), 'Sec')
# print('Median Comment time: ',round(np.median(ac[1:-1]),2), 'Sec')
# print('Median Answer time: ',round(np.median(aa[1:-1])/60,2), 'Min')
# print('Median Comment time: ',round(np.median(ac[1:-1])/60,2), 'Min')
print('Median Answer time: ',round(np.median(aa[1:-1])/3600,2), 'Hr')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600,2), 'Hr')
print('S: ',round(ps,2))
print('A: ',round(pa,2))
print('C: ',round(pc,2))
print('F: ',round(pf,2))
print('V: ',round(pv,2))
print('P: ',round(pp,2))

print("Answer responses: ",len(aa))
Q1 = np.percentile(aa, 25,
                   interpolation = 'midpoint')
 
Q3 = np.percentile(aa, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1
# Upper bound
upper = np.where(aa >= (Q3+1.5*IQR))
# Lower bound
lower = np.where(aa <= (Q1-1.5*IQR))
print("outlier: " ,len(upper[0])+len(lower[0]))


Theme: Device
Q:  221
Freq: 22.44 %
A:  270
A/Q:  1.22
Without an accpeted answer:  65.16 %
Unsuccessful:  22.62 %
Median Answer time:  4.53 Hr
S:  18.46
A:  6.92
C:  14.26
F:  2.11
V:  4.52
P:  46.27
Answer responses:  77
outlier:  12


/var/folders/m4/q4g0vczj17dcpz2pllsf7c_m0000gn/T/ipykernel_44651/2481741884.py:61: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  Q1 = np.percentile(aa, 25,
/var/folders/m4/q4g0vczj17dcpz2pllsf7c_m0000gn/T/ipykernel_44651/2481741884.py:64: DeprecationWarning: the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  Q3 = np.percentile(aa, 75,


In [17]:
print('Theme: Monitoring')

row = data[data['Monitoring']>=thrs]
# row = data[data['Theme'].str.lower().str.contains('measur') | data['Tags'].str.lower().str.contains('measur')]

print('Q: ',len(row))
print('Freq:', round(len(row)/985*100,2), '%')
a =0
s,u=0,0
aa,ac = [],[]
ps,pa,pc,pf,pv,pp = 0.0,0.0,0.0,0.0,0.0,0.0
for index, e in row.iterrows():
    ps += pop[e[0]]['s']
    pa += pop[e[0]]['a']
    pc += pop[e[0]]['c']
    pf += pop[e[0]]['f']
    pv += pop[e[0]]['v']
    pp += pop[e[0]]['p']
    a += int(e[6+6])
    if e[6+6] == 0: #AnswerCount
        u+=1
    elif e[-2] != '': #AnswerDate
        s+=1
        #AnswerDate-QuestionDate
        aa.append((datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
    if e[-4] != '': #CommentDate
        if e[-2] != '':
            #comment date before the date the author accpets the answer
            #CommentDate-AnswerDate
            if (datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds()<0:
                #CommentDate-QuestionDate
                ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
        else:
            #CommentDate-QuestionDate
            ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())

print('A: ',a)
print('A/Q: ', round(a/len(row),2))
print('Without an accpeted answer: ',round(100-s/len(row)*100,2), '%')
# print('Successful: ',round(s/len(row)*100,2), '%')
# print('Ordinary: ',round((len(row)-u-s)/len(row)*100,2), '%')
print('Unsuccessful: ',round(u/len(row)*100,2), '%')
# print('Answered: ',round((len(row)-u)/len(row)*100,2), '%')
aa = sorted(aa, key=lambda x:x, reverse=True)
ac = sorted(ac, key=lambda x:x, reverse=True)
#remove the highest and lowest
# print('Median Answer time: ',round(np.median(aa[1:-1]),2), 'days')
# print('Median Comment time: ',round(np.median(ac[1:-1]),2), 'days')
print('Median Answer time: ',round(np.median(aa[1:-1])/3600,2), 'Hr')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600,2), 'Hr')
print('S: ',round(ps,2))
print('A: ',round(pa,2))
print('C: ',round(pc,2))
print('F: ',round(pf,2))
print('V: ',round(pv,2))
print('P: ',round(pp,2))
print("Answer responses: ",len(aa))

Theme: Monitoring
Q:  207
Freq: 21.02 %
A:  272
A/Q:  1.31
Without an accpeted answer:  63.29 %
Unsuccessful:  16.91 %
Median Answer time:  5.74 Hr
S:  18.58
A:  7.33
C:  12.35
F:  3.14
V:  6.31
P:  47.71
Answer responses:  76


In [14]:
print('Theme: Computing Source')
row = data[data['CS']>=thrs]
# row = data[data['Theme'].str.lower().str.contains('mode') | data['Tags'].str.lower().str.contains('mode')]

print('Q: ',len(row))
print('Freq:', round(len(row)/985*100,2), '%')
a =0
s,u=0,0
aa,ac = [],[]
ps,pa,pc,pf,pv,pp = 0.0,0.0,0.0,0.0,0.0,0.0
for index, e in row.iterrows():
    ps += pop[e[0]]['s']
    pa += pop[e[0]]['a']
    pc += pop[e[0]]['c']
    pf += pop[e[0]]['f']
    pv += pop[e[0]]['v']
    pp += pop[e[0]]['p']
    a += int(e[6+6])
    if e[6+6] == 0: #AnswerCount
        u+=1
    elif e[-2] != '': #AnswerDate
        s+=1
        #AnswerDate-QuestionDate
        aa.append((datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
    if e[-4] != '': #CommentDate
        if e[-2] != '':
            #comment date before the date the author accpets the answer
            #CommentDate-AnswerDate
            if (datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds()<0:
                #CommentDate-QuestionDate
                ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
        else:
            #CommentDate-QuestionDate
            ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())

print('A: ',a)
print('A/Q: ', round(a/len(row),2))
print('Without an accpeted answer: ',round(100-s/len(row)*100,2), '%')
# print('Successful: ',round(s/len(row)*100,2), '%')
# print('Ordinary: ',round((len(row)-u-s)/len(row)*100,2), '%')
print('Unsuccessful: ',round(u/len(row)*100,2), '%')
# print('Answered: ',round((len(row)-u)/len(row)*100,2), '%')
aa = sorted(aa, key=lambda x:x, reverse=True)
ac = sorted(ac, key=lambda x:x, reverse=True)
#remove the highest and lowest
# print('Median Answer time: ',round(np.median(aa[1:-1]),2), 'days')
# print('Median Comment time: ',round(np.median(ac[1:-1]),2), 'days')
print('Median Answer time: ',round(np.median(aa[1:-1])/3600,2), 'Hr')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600,2), 'Hr')
print('S: ',round(ps,2))
print('A: ',round(pa,2))
print('C: ',round(pc,2))
print('F: ',round(pf,2))
print('V: ',round(pv,2))
print('P: ',round(pp,2))
print("Answer responses: ",len(aa))


Theme: Computing Source
Q:  272
Freq: 27.61 %
A:  376
A/Q:  1.38
Without an accpeted answer:  59.93 %
Unsuccessful:  22.79 %
Median Answer time:  5.47 Hr
S:  25.68
A:  11.1
C:  27.74
F:  4.0
V:  5.2
P:  73.72
Answer responses:  109


In [15]:
print('Theme: Data Transmission')

row = data[data['DT']>=thrs]

print('Q: ',len(row))
print('Freq:', round(len(row)/985*100,2), '%')
a =0
s,u=0,0
aa,ac = [],[]
ps,pa,pc,pf,pv,pp = 0.0,0.0,0.0,0.0,0.0,0.0
for index, e in row.iterrows():
    ps += pop[e[0]]['s']
    pa += pop[e[0]]['a']
    pc += pop[e[0]]['c']
    pf += pop[e[0]]['f']
    pv += pop[e[0]]['v']
    pp += pop[e[0]]['p']
    a += int(e[6+6])
    if e[6+6] == 0: #AnswerCount
        u+=1
    elif e[-2] != '': #AnswerDate
        s+=1
        #AnswerDate-QuestionDate
        aa.append((datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
    if e[-4] != '': #CommentDate
        if e[-2] != '':
            #comment date before the date the author accpets the answer
            #CommentDate-AnswerDate
            if (datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[-2].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds()<0:
                #CommentDate-QuestionDate
                ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())
        else:
            #CommentDate-QuestionDate
            ac.append((datetime.datetime.strptime(e[-4].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")-datetime.datetime.strptime(e[3].replace('T', ' ').split('.')[0], "%Y-%m-%d %H:%M:%S")).total_seconds())

print('A: ',a)
print('A/Q: ', round(a/len(row),2))
print('Without an accpeted answer: ',round(100-s/len(row)*100,2), '%')
# print('Successful: ',round(s/len(row)*100,2), '%')
# print('Ordinary: ',round((len(row)-u-s)/len(row)*100,2), '%')
print('Unsuccessful: ',round(u/len(row)*100,2), '%')
# print('Answered: ',round((len(row)-u)/len(row)*100,2), '%')
aa = sorted(aa, key=lambda x:x, reverse=True)
ac = sorted(ac, key=lambda x:x, reverse=True)
#remove the highest and lowest
# print('Median Answer time: ',round(np.median(aa[1:-1]),2), 'days')
# print('Median Comment time: ',round(np.median(ac[1:-1]),2), 'days')
print('Median Answer time: ',round(np.median(aa[1:-1])/3600,2), 'Hr')
# print('Median Comment time: ',round(np.median(ac[1:-1])/3600,2), 'Hr')
print('S: ',round(ps,2))
print('A: ',round(pa,2))
print('C: ',round(pc,2))
print('F: ',round(pf,2))
print('V: ',round(pv,2))
print('P: ',round(pp,2))
print("Answer responses: ",len(aa))


Theme: Data Transmission
Q:  125
Freq: 12.69 %
A:  175
A/Q:  1.4
Without an accpeted answer:  56.8 %
Unsuccessful:  21.6 %
Median Answer time:  2.43 Hr
S:  11.44
A:  4.99
C:  6.48
F:  1.6
V:  2.78
P:  27.29
Answer responses:  54


In [19]:
db.close()
# c.close()